# Demo usage of the information from the TMVA interface fitting  TPC QA variables
* Load TMVA interface function
* Load tree and defining derived information (TTree aliases) and metadata
* CacheTree input variables to tree format usable by TMVA
* Register example methods used for regression
* Emulation of the bootstrap - training repeated several time
* Load array of regression -used later in the array regression evaluation (mean, median, rms)
* Example draw queries

In [1]:
%jsroot

In [2]:
gSystem->AddIncludePath("-I$ALICE_ROOT/include/"); //couldn't add include path in .rootr
AliDrawStyle::SetDefaults();
AliDrawStyle::ApplyStyle("figTemplate");
TCanvas *canvasDraw = new TCanvas("canvasDraw","canvasDraw",900,700);
TTree *treeCache=0,*tree=0;

Info in <AliDrawStyle::ApplyStyle>: figTemplate


### Load TMVA interface function

In [3]:
.L $AliRoot_SRC/STAT/Macros/AliNDFunctionInterface.cxx+

### Load tree and defining derived  information (TTree aliases)  and metadata

In [4]:
AliExternalInfo info;
tree = info.GetChain("QA.TPC", "LHC17*", "cpass1_pass1", "QA.EVS;QA.rawTPC");
tree->SetAlias("interactionRate", "QA.EVS.interactionRate");
///
tree->SetAlias("qmaxQASum", "Sum$(qmaxQA.fElements*((abs(qmaxQA.fElements-40)<20)))/Sum$((abs(qmaxQA.fElements-40)<20))");
tree->SetAlias("qmaxQASumIn", "Sum$(qmaxQA.fElements*((Iteration$<36&&abs(qmaxQA.fElements-40)<20)))/Sum$((Iteration$<36&&abs(qmaxQA.fElements-40)<20))");
tree->SetAlias("qmaxQASumOut", "Sum$(qmaxQA.fElements*((Iteration$>=36&&abs(qmaxQA.fElements-40)<20)))/Sum$((Iteration$>=36&&abs(qmaxQA.fElements-40)<20))");
tree->SetAlias("qmaxQASumR", "qmaxQASumIn/qmaxQASum");
tree->SetAlias("meanMIPeleR", "meanMIPele/meanMIP");
tree->SetAlias("bz0", "bz+rndm*0.0001");
tree->SetMarkerStyle(21); tree->SetMarkerSize(0.5);

Info in <AliExternalInfo::ReadConfig>: Path: $ALICE_ROOT/STAT/Macros/AliExternalInfo.cfg	/data/alicesw6/sw/ubuntu1604_x86-64/AliRoot/0_ROOT6-1/STAT/Macros/AliExternalInfo.cfg
Info in <AliExternalInfo::SetupVariables>: Information will be stored/retrieved in/from /homeold/miranov/AliExternalInfoCache//data/2017/LHC17*/cpass1_pass1/
Info in <AliExternalInfo::GetChain>: Files to add to chain: /homeold/miranov/AliExternalInfoCache//data/2017/LHC17c/cpass1_pass1/TPC_trending.root
/homeold/miranov/AliExternalInfoCache//data/2017/LHC17e/cpass1_pass1/TPC_trending.root
/homeold/miranov/AliExternalInfoCache//data/2017/LHC17f/cpass1_pass1/TPC_trending.root
/homeold/miranov/AliExternalInfoCache//data/2017/LHC17g/cpass1_pass1/TPC_trending.root
/homeold/miranov/AliExternalInfoCache//data/2017/LHC17h/cpass1_pass1/TPC_trending.root
/homeold/miranov/AliExternalInfoCache//data/2017/LHC17i/cpass1_pass1/TPC_trending.root
/homeold/miranov/AliExternalInfoCache//data/2017/LHC17j/cpass1_pass1/TPC_trending.roo

Collection name='metaTable', class='THashList', size=1490
 OBJ: TNamed	DCAr_Status.html	%d<run>/dca_and_phi.png
 OBJ: TNamed	DCAz_Status.html	%d<run>/dca_and_phi.png
 OBJ: TNamed	MIPattachSlopeA.AxisTitle	 dEdx(MIP/50) Attach
 OBJ: TNamed	MIPattachSlopeA.Description	TPC standard QA variables.  Class TPC dEdx Attach ASide
 OBJ: TNamed	MIPattachSlopeA.Legend	 dEdx Attach A side
 OBJ: TNamed	MIPattachSlopeA.Title	 dEdx Attach A side
 OBJ: TNamed	MIPattachSlopeA.class	TPC dEdx Attach ASide
 OBJ: TNamed	MIPattachSlopeC.AxisTitle	 dEdx(MIP/50) Attach
 OBJ: TNamed	MIPattachSlopeC.Description	TPC standard QA variables.  Class TPC dEdx Attach CSide
 OBJ: TNamed	MIPattachSlopeC.Legend	 dEdx Attach C side
 OBJ: TNamed	MIPattachSlopeC.Title	 dEdx Attach C side
 OBJ: TNamed	MIPattachSlopeC.class	TPC dEdx Attach CSide
 OBJ: TNamed	PID_Status.html	%d<run>/TPC_dEdx_track_info.png
 OBJ: TNamed	TPC_Occ_IROC..AxisTitle	 Occupancy
 OBJ: TNamed	TPC_Occ_IROC..Description	TPC standard QA variables.  Class TP

Info in <qatpcAddMetadata>: Start processing Tree tpcQA
Info in <qatpcAddMetadata>: End
Info in <AliExternalInfo::SetupVariables>: Information will be stored/retrieved in/from /homeold/miranov/AliExternalInfoCache//data/2017/LHC17*/cpass1_pass1/
Info in <AliExternalInfo::GetChain>: Files to add to chain: /homeold/miranov/AliExternalInfoCache//data/2017/LHC17c/cpass1_pass1/EVS_trending.root
/homeold/miranov/AliExternalInfoCache//data/2017/LHC17d/cpass1_pass1/EVS_trending.root
/homeold/miranov/AliExternalInfoCache//data/2017/LHC17e/cpass1_pass1/EVS_trending.root
/homeold/miranov/AliExternalInfoCache//data/2017/LHC17f/cpass1_pass1/EVS_trending.root
/homeold/miranov/AliExternalInfoCache//data/2017/LHC17g/cpass1_pass1/EVS_trending.root
/homeold/miranov/AliExternalInfoCache//data/2017/LHC17h/cpass1_pass1/EVS_trending.root
/homeold/miranov/AliExternalInfoCache//data/2017/LHC17i/cpass1_pass1/EVS_trending.root
/homeold/miranov/AliExternalInfoCache//data/2017/LHC17j/cpass1_pass1/EVS_trending.roo

## CacheTree input variables to tree format usable by TMVA
-------------------------------
* TMVA can not work with friend trees with indeces, respec. with array of the measurements, resp aliases and functions
  * input "flat tree" for MVA learning to be created with variables of interest
  * AliTreePlayer::MakeCacheTree to create flat input tree for TMVA
  ````
  AliTreePlayer::MakeCacheTree(tree,"resolutionMIP:meanMIPeleR:tpcItsMatchA:bz0:interactionRate:qmaxQASum:qmaxQASumIn:qmaxQASumOut:qmaxQASumR:run:time","TMVAInput.root","MVAInput","meanMIP>30&&run==QA.EVS.run");
````

In [5]:
AliTreePlayer::MakeCacheTree(tree,"resolutionMIP:meanMIPeleR:tpcItsMatchA:bz0:interactionRate:qmaxQASum:qmaxQASumIn:qmaxQASumOut:qmaxQASumR:run:time","TMVAInput.root","MVAInput","meanMIP>30&&run==QA.EVS.run");

### Register example methods used for regression
----------------------------------------
* BDT and MLP example
* DNN - for the moment not used as need BLASS library  - not in default AliRoot
   * Naive adaptation of the DNN configuration from the ROOT tutorials  TMVARegression.C -
   * slow 100 times smaller than MLP
   * lead to floating point exception
   

In [6]:
TString layoutString("Layout=TANH|20,LINEAR");
  TString training0("LearningRate=1e-5,Momentum=0.5,Repetitions=1,ConvergenceSteps=500,BatchSize=50,"
                    "TestRepetitions=7,WeightDecay=0.01,Regularization=L1,DropConfig=0.5+0.5+0.5+0.5,"
                    "DropRepetitions=2");
  TString training1("LearningRate=1e-5,Momentum=0.9,Repetitions=1,ConvergenceSteps=170,BatchSize=30,"
                    "TestRepetitions=7,WeightDecay=0.01,Regularization=L1,DropConfig=0.1+0.1+0.1,DropRepetitions="
                    "1");
  TString trainingStrategyString("TrainingStrategy=");
  trainingStrategyString += training0 + "|" + training1;
  TString dnnOptions("!H:V:ErrorStrategy=SUMOFSQUARES:VarTransform=G:WeightInitialization=XAVIERUNIFORM:Architecture=CPU");
  dnnOptions.Append(":");
  dnnOptions.Append(layoutString);
  dnnOptions.Append(":");
  dnnOptions.Append(trainingStrategyString);
  ///
  AliNDFunctionInterface::registerMethod("BDTRF25_8","!H:!V:NTrees=25:Shrinkage=0.1:UseRandomisedTrees:nCuts=20:MaxDepth=8",TMVA::Types::kBDT);
  AliNDFunctionInterface::registerMethod("BDTRF12_16", "!H:!V:NTrees=12:Shrinkage=0.1:UseRandomisedTrees:nCuts=20:MaxDepth=16", TMVA::Types::kBDT);
  AliNDFunctionInterface::registerMethod("KNN","nkNN=20:ScaleFrac=0.8:SigmaFact=1.0:Kernel=Gaus:UseKernel=F:UseWeight=T:!Trim", TMVA::Types::kKNN);
  AliNDFunctionInterface::registerMethod("MLP", "!H:!V:VarTransform=Norm:NeuronType=tanh:NCycles=20000:HiddenLayers=N+20:TestRate=6:TrainingMethod=BFGS:Sampling=0.3:SamplingEpoch=0.8:ConvergenceImprove=1e-6:ConvergenceTests=15:!UseRegulator",TMVA::Types::kMLP);
  AliNDFunctionInterface::registerMethod("DNN_CPU",dnnOptions.Data(), TMVA::Types::kDNN);

### Emulation of the bootstrap - training repeated several time
* TODO - Implement real bootstrap ( random sampling with replacement + other methods) in the TMVA (to check with ROOT)
* TODO - DNN example - TO USE DNN (Deep neural network)  BLASS or CBLASS library has to be enabled in ROOT. This is not the case for the default aliBuild recipies

In [7]:
gSystem->Exec("rm -f TMVA_RegressionOutput.root");
TFile *f= TFile::Open("TMVAInput.root");
treeCache=0;
nRegression=10;
f->GetObject("MVAInput",treeCache);
for (Int_t iBoot=0; iBoot<nRegression; iBoot++) {
    AliNDFunctionInterface::FitMVA(treeCache, "resolutionMIP", "interactionRate>0", "interactionRate:bz0:qmaxQASum:qmaxQASumR", "BDTRF25_8:BDTRF12_16:KNN", 0, iBoot);
     AliNDFunctionInterface::FitMVA(treeCache, "meanMIPeleR", "interactionRate>0", "interactionRate:bz0:qmaxQASum:qmaxQASumR", "BDTRF25_8:BDTRF12_16:KNN", 0, iBoot);
     AliNDFunctionInterface::FitMVA(treeCache, "tpcItsMatchA", "interactionRate>0", "interactionRate:bz0:qmaxQASum:qmaxQASumR", "BDTRF25_8:BDTRF12_16:KNN", 0, iBoot);
}

DataSetInfo              : [dir_resolutionMIP[0]] : Added class "Regression"
                         : Add Tree MVAInput of type Regression with 851 events
                         : Dataset[dir_resolutionMIP[0]] : Class index : 0  name : Regression
Booking method BDTRF25_8	9	!H:!V:NTrees=25:Shrinkage=0.1:UseRandomisedTrees:nCuts=20:MaxDepth=8
Factory                  : Booking method: BDTRF25_8
                         : 
DataSetFactory           : [dir_resolutionMIP[0]] : Number of events in input trees
                         : Dataset[dir_resolutionMIP[0]] :     Regression requirement: "interactionRate>0"
                         : Dataset[dir_resolutionMIP[0]] :     Regression      -- number of events passed: 849    / sum of weights: 849  
                         : Dataset[dir_resolutionMIP[0]] :     Regression      -- efficiency             : 0.99765
                         : Dataset[dir_resolutionMIP[0]] :  you have opted for interpreting the requested number of training/tes

4%, time left: unknown
12%, time left: 0 sec
20%, time left: 0 sec
24%, time left: 0 sec
32%, time left: 0 sec
36%, time left: 0 sec
44%, time left: 0 sec
48%, time left: 0 sec
56%, time left: 0 sec
64%, time left: 0 sec
68%, time left: 0 sec
76%, time left: 0 sec
80%, time left: 0 sec
88%, time left: 0 sec
92%, time left: 0 sec
100%, time left: 0 sec
0%, time left: unknown
6%, time left: 0 sec
13%, time left: 0 sec
19%, time left: 0 sec
25%, time left: 0 sec
32%, time left: 0 sec
37%, time left: 0 sec
44%, time left: 0 sec
50%, time left: 0 sec
56%, time left: 0 sec
63%, time left: 0 sec
69%, time left: 0 sec
75%, time left: 0 sec
82%, time left: 0 sec
87%, time left: 0 sec
94%, time left: 0 sec


Factory                  : Training finished
                         : 
Factory                  : Train method: BDTRF12_16 for Regression
                         : 
                         : Regression Loss Function: Huber
                         : Training 12 Decision Trees ... patience please
                         : Elapsed time for training with 424 events: 0.0562 sec         
                         : Dataset[dir_resolutionMIP[0]] : Create results for training
                         : Dataset[dir_resolutionMIP[0]] : Evaluation of BDTRF12_16 on training sample
                         : Dataset[dir_resolutionMIP[0]] : Elapsed time for evaluation of 424 events: 0.00167 sec       
                         : Create variable histograms
                         : Create regression target histograms
                         : Create regression average deviation
                         : Results created
                         : Creating xml weight file: dir_resolutionMIP[0]/w

8%, time left: unknown
16%, time left: 0 sec
25%, time left: 0 sec
33%, time left: 0 sec
41%, time left: 0 sec
50%, time left: 0 sec
58%, time left: 0 sec
66%, time left: 0 sec
75%, time left: 0 sec
83%, time left: 0 sec
91%, time left: 0 sec
100%, time left: 0 sec
0%, time left: unknown
6%, time left: 0 sec
13%, time left: 0 sec
19%, time left: 0 sec
25%, time left: 0 sec
32%, time left: 0 sec
37%, time left: 0 sec
44%, time left: 0 sec
50%, time left: 0 sec
56%, time left: 0 sec
63%, time left: 0 sec
69%, time left: 0 sec
75%, time left: 0 sec
82%, time left: 0 sec
87%, time left: 0 sec
94%, time left: 0 sec
0%, time left: unknown
6%, time left: 0 sec
13%, time left: 0 sec
19%, time left: 0 sec
25%, time left: 0 sec
32%, time left: 0 sec
37%, time left: 0 sec
44%, time left: 0 sec
50%, time left: 0 sec
56%, time left: 0 sec
63%, time left: 0 sec
69%, time left: 0 sec
75%, time left: 0 sec
82%, time left: 0 sec
87%, time left: 0 sec
94%, time left: 0 sec


                         : Reading weight file: dir_resolutionMIP[0]/weights/TMVARegression_BDTRF12_16.weights.xml
                         : Reading weight file: dir_resolutionMIP[0]/weights/TMVARegression_KNN.weights.xml
                         : Creating kd-tree with 424 events
                         : Computing scale factor for 1d distributions: (ifrac, bottom, top) = (80%, 10%, 90%)
ModulekNN                : Optimizing tree for 4 variables with 424 values
                         : <Fill> Class 1 has      424 events
Weight file	dir_resolutionMIP[0]/weights/TMVARegression_BDTRF25_8.weights.xml
Weight file	dir_resolutionMIP[0]/weights/TMVARegression_BDTRF12_16.weights.xml
Weight file	dir_resolutionMIP[0]/weights/TMVARegression_KNN.weights.xml
Factory                  : Test all methods
Factory                  : Test method: BDTRF25_8 for Regression performance
                         : 
                         : Dataset[dir_resolutionMIP[0]] : Create results for testing
     

...

0%, time left: unknown
6%, time left: 0 sec
13%, time left: 0 sec
19%, time left: 0 sec
25%, time left: 0 sec
32%, time left: 0 sec
37%, time left: 0 sec
44%, time left: 0 sec
50%, time left: 0 sec
56%, time left: 0 sec
63%, time left: 0 sec
69%, time left: 0 sec
75%, time left: 0 sec
82%, time left: 0 sec
87%, time left: 0 sec
94%, time left: 0 sec


Weight file	dir_tpcItsMatchA[9]/weights/TMVARegression_BDTRF12_16.weights.xml
Weight file	dir_tpcItsMatchA[9]/weights/TMVARegression_KNN.weights.xml
Factory                  : Test all methods
Factory                  : Test method: BDTRF25_8 for Regression performance
                         : 
                         : Dataset[dir_tpcItsMatchA[9]] : Create results for testing
                         : Dataset[dir_tpcItsMatchA[9]] : Evaluation of BDTRF25_8 on testing sample
                         : Dataset[dir_tpcItsMatchA[9]] : Elapsed time for evaluation of 424 events: 0.00366 sec       
                         : Create variable histograms
                         : Create regression target histograms
                         : Create regression average deviation
                         : Results created
Factory                  : Test method: BDTRF12_16 for Regression performance
                         : 
                         : Dataset[dir_tpcItsMatchA[9]] : Create res

0%, time left: unknown
6%, time left: 0 sec
13%, time left: 0 sec
19%, time left: 0 sec
25%, time left: 0 sec
32%, time left: 0 sec
37%, time left: 0 sec
44%, time left: 0 sec
50%, time left: 0 sec
56%, time left: 0 sec
63%, time left: 0 sec
69%, time left: 0 sec
75%, time left: 0 sec
82%, time left: 0 sec
87%, time left: 0 sec
94%, time left: 0 sec
0%, time left: unknown
6%, time left: 0 sec
13%, time left: 0 sec
19%, time left: 0 sec
25%, time left: 0 sec
32%, time left: 0 sec
37%, time left: 0 sec
44%, time left: 0 sec
50%, time left: 0 sec
56%, time left: 0 sec
63%, time left: 0 sec
69%, time left: 0 sec
75%, time left: 0 sec
82%, time left: 0 sec
87%, time left: 0 sec
94%, time left: 0 sec
0%, time left: unknown
6%, time left: 0 sec
13%, time left: 0 sec
19%, time left: 0 sec
25%, time left: 0 sec
32%, time left: 0 sec
37%, time left: 0 sec
44%, time left: 0 sec
50%, time left: 0 sec
56%, time left: 0 sec
63%, time left: 0 sec
69%, time left: 0 sec
75%, time left: 0 sec
82%, time 

TFHandler_BDTRF12_16     :        Variable               Mean               RMS       [        Min               Max ]
                         : ----------------------------------------------------------------------------------------------
                         : interactionRate:       1.6417e+05           65163.   [           135.79       7.5972e+05 ]
                         :             bz0:         -0.46929          0.15738   [         -0.50000          0.50010 ]
                         :       qmaxQASum:           39.441           2.6337   [           32.944           43.396 ]
                         :      qmaxQASumR:          0.95476         0.019165   [          0.85734          0.97138 ]
                         :    tpcItsMatchA:          0.73669         0.068967   [          0.49744          0.93753 ]
                         : ----------------------------------------------------------------------------------------------
                         : Evaluate regression 

0%, time left: unknown
60%, time left: 0 sec
100%, time left: 0 sec
0%, time left: unknown
60%, time left: 0 sec
100%, time left: 0 sec


### Load array of regression  -used later in the array regression evaluation (mean, median, rms)

In [8]:
/// Load array of regression  -used later in the array regression evaluation (mean, median, rms)
///-------------------------------
  AliNDFunctionInterface::LoadMVAReaderArray(0,"TMVA_RegressionOutput.root","BDTRF12_16",".*resolutionMIP");
  AliNDFunctionInterface::LoadMVAReaderArray(1,"TMVA_RegressionOutput.root","BDTRF25_8",".*resolutionMIP");
  AliNDFunctionInterface::LoadMVAReaderArray(2,"TMVA_RegressionOutput.root","KNN",".*resolutionMIP");

dir_resolutionMIP[0]
dir_resolutionMIP[0]/BDTRF12_16
interactionRate
bz0
qmaxQASum
qmaxQASumR
                         : Booking "BDTRF12_16" of type "BDT" from weights.xml.
                         : Reading weight file: weights.xml
<HEADER> DataSetInfo              : [Default] : Added class "Regression"
                         : Booked classifier "BDTRF12_16" of type: "BDT"
dir_resolutionMIP[1]
dir_resolutionMIP[1]/BDTRF12_16
interactionRate
bz0
qmaxQASum
qmaxQASumR
                         : Booking "BDTRF12_16" of type "BDT" from weights.xml.
                         : Reading weight file: weights.xml
<HEADER> DataSetInfo              : [Default] : Added class "Regression"
                         : Booked classifier "BDTRF12_16" of type: "BDT"
dir_resolutionMIP[2]
dir_resolutionMIP[2]/BDTRF12_16
interactionRate
bz0
qmaxQASum
qmaxQASumR
                         : Booking "BDTRF12_16" of type "BDT" from weights.xml.
                         : Reading weight file: weights.xml
<HEADE

In [9]:
tree->Draw("AliNDFunctionInterface::EvalMVAStat(0,1,interactionRate, bz0,qmaxQASum,qmaxQASumR):resolutionMIP:run","run==QA.EVS.run","colz");
canvasDraw->Draw("colz");

In [10]:
 tree->Draw("AliNDFunctionInterface::EvalMVAStat(0,1,interactionRate, bz0,qmaxQASum,qmaxQASumR)/resolutionMIP:run:interactionRate","run==QA.EVS.run","colz");
canvasDraw->Draw("colz");

In [11]:
  tree->Draw("AliNDFunctionInterface::EvalMVAStat(0,2,interactionRate, bz0,qmaxQASum,qmaxQASumR):AliNDFunctionInterface::EvalMVAStat(1,2,interactionRate, bz0,qmaxQASum,qmaxQASumR)","run==QA.EVS.run","colz");
canvasDraw->Draw("colz");